In [1]:
# Importing libraries and functions
import os
import string
from string import digits
import re
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
df = pd.read_csv('EnHi.csv')

In [3]:
df.head()

,SourceName,TargetName/0/_ID,TargetName/0/__text,_ID,TargetName/1/_ID,TargetName/1/__text,TargetName/2/_ID,TargetName/2/__text
0,RAASAVIHAAREE,1,रासविहारी,1,NaN,NaN,NaN,NaN
1,DEOGAN ROAD,1,देवगन रोड,2,NaN,NaN,NaN,NaN
2,SHATRUMARDAN,1,शत्रुमर्दन,3,NaN,NaN,NaN,NaN
3,MAHIJUBA,1,महिजुबा,4,NaN,NaN,NaN,NaN
4,SABINE,1,सैबिन,5,NaN,NaN,NaN,NaN


In [4]:
tname = df['TargetName/0/__text']
df['TargetName'] = tname

In [5]:
#cleaning data
df = df.drop(['TargetName/0/_ID', 'TargetName/0/__text', '_ID',	'TargetName/1/_ID', 'TargetName/1/__text',	'TargetName/2/_ID', 'TargetName/2/__text'], axis = 1)
df['SourceName'] = df['SourceName'].apply(lambda x: x.lower())
df['TargetName'] = df['TargetName'].apply(lambda x: x.lower())
df.head()

,SourceName,TargetName
0,raasavihaaree,रासविहारी
1,deogan road,देवगन रोड
2,shatrumardan,शत्रुमर्दन
3,mahijuba,महिजुबा
4,sabine,सैबिन


In [6]:
new_df = pd.DataFrame({'SourceName':df['TargetName'],'TargetName':df['SourceName']})
df = new_df.copy()
df.head()

,SourceName,TargetName
0,रासविहारी,raasavihaaree
1,देवगन रोड,deogan road
2,शत्रुमर्दन,shatrumardan
3,महिजुबा,mahijuba
4,सैबिन,sabine


In [7]:
#adding start and end tokens
#start token - '^'
#end token - '!'
hin = []
mx_tr = -1
for i in df['TargetName']:
  tmp = i
  tmp = '^' + tmp + '!'
  mx_tr = max(mx_tr,len(tmp))
  hin.append(tmp)

df['TargetName'] = hin
df.head()

,SourceName,TargetName
0,रासविहारी,^raasavihaaree!
1,देवगन रोड,^deogan road!
2,शत्रुमर्दन,^shatrumardan!
3,महिजुबा,^mahijuba!
4,सैबिन,^sabine!


In [8]:
# counting unique letters in English
d = {}
mx_sr = -1
for i in df['TargetName']:
  mx_sr = max(mx_sr,len(i))
  l = [item for item in i]
  for j in l:
    if(j not in d):
      d[j] = 0
    if(j in d):
      d[j] += 1

In [9]:
# counting uniaue letters in Hindi
e = {}
for i in df['SourceName']:
  l = [item for item in i.lower()]
  for j in l:
    if(j not in e):
      e[j] = 0
    if(j in e):
      e[j] += 1

In [10]:
max_length_src=mx_sr
max_length_tar=mx_tr
input_words = sorted(list(e.keys()))
target_words = sorted(list(d.keys()))
num_encoder_tokens = len(e)
num_decoder_tokens = len(d)
num_encoder_tokens, num_decoder_tokens

(83, 45)

In [11]:
num_decoder_tokens += 1
num_encoder_tokens += 1

In [12]:
# preparing tokens for each letters
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [13]:
X, y = df['SourceName'], df['TargetName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state=0)
X_train.shape, X_test.shape

((12543,), (1394,))

In [14]:
# generating train data using one hot encoding
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the '^' (Start) token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [15]:
# Encoder network
latent_dim = 100
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [16]:
# Decoder Network
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [18]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 3

In [19]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
391/391 [==============================] - 62s 157ms/step - loss: 0.4474 - val_loss: 0.4101
Epoch 2/3
391/391 [==============================] - 60s 154ms/step - loss: 0.3733 - val_loss: 0.3008
Epoch 3/3
391/391 [==============================] - 60s 155ms/step - loss: 0.3018 - val_loss: 0.2280


In [20]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [21]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['^']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '!' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [22]:
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k = 0
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Hindi Word:', X_test[k:k+1].values[0])
print('Predicted English Word:', decoded_sentence[:-1])

Hindi Word: सभ्य
Predicted English Word:  s h a b a 


In [23]:
k = 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Hindi Word:', X_test[k:k+1].values[0])
print('Predicted English Word:', decoded_sentence[:-1])

Hindi Word: प्रतापगढ़
Predicted English Word:  p r a t t a r a d 
